# Spatial data, denormalisation and ORM

``<jeep@cphbusiness.dk>``

## Agenda

* Spatial data
* Denormalisation
* PL/pgSQL
* Object-relational mapping

### Literature

* http://www.cs.toronto.edu/~faye/343/f07/lectures/wk12/wk12_BCNF2-up.pdf

In [ ]:
%load_ext sql

In [ ]:
%matplotlib inline

In [ ]:
%sql postgresql://appdev@0.0.0.0/appdev

## Solution to assignment

In [ ]:
%%sql
SELECT driverref, name, count(*), sum(milliseconds) / count(*)
FROM results
INNER JOIN (SELECT driverid, raceid, sum(milliseconds) FROM pitstops GROUP BY driverid, raceid) AS p using (driverid, raceid) 
INNER JOIN drivers using (driverid) 
INNER JOIN constructors using (constructorid)
WHERE statusid = 1
group by constructorid, name, driverid, driverref ORDER BY sum(milliseconds) / count(*) ASC;

## Solution with view

In [ ]:
%%sql
CREATE VIEW pitstop_view AS 
  SELECT driverid, raceid, sum(milliseconds) FROM pitstops GROUP BY driverid, raceid

In [ ]:
%%sql
SELECT driverref, name, count(*), sum(milliseconds) / count(*)
FROM results
INNER JOIN pitstop_view using (driverid, raceid) 
INNER JOIN drivers using (driverid) 
INNER JOIN constructors using (constructorid)
WHERE statusid = 1
group by constructorid, name, driverid, driverref ORDER BY sum(milliseconds) / count(*) ASC;

# Handling spatial data in PostgreSQL

* Geometric data types
* Geometric functions

## Geometric data types

https://www.postgresql.org/docs/current/static/datatype-geometric.html

<table class="CALSTABLE" border="1">
      <colgroup><col>
      <col>
      <col>
      <col>
      </colgroup><thead>
        <tr>
          <th>Name</th>
          <th>Storage Size</th>
          <th>Description</th>
          <th>Representation</th>
        </tr>
      </thead>
      <tbody>
        <tr>
          <td><tt class="TYPE">point</tt></td>
          <td>16 bytes</td>
          <td>Point on a plane</td>
          <td>(x,y)</td>
        </tr>
        <tr>
          <td><tt class="TYPE">line</tt></td>
          <td>32 bytes</td>
          <td>Infinite line</td>
          <td>{A,B,C}</td>
        </tr>
        <tr>
          <td><tt class="TYPE">lseg</tt></td>
          <td>32 bytes</td>
          <td>Finite line segment</td>
          <td>((x1,y1),(x2,y2))</td>
        </tr>
        <tr>
          <td><tt class="TYPE">box</tt></td>
          <td>32 bytes</td>
          <td>Rectangular box</td>
          <td>((x1,y1),(x2,y2))</td>
        </tr>
        <tr>
          <td><tt class="TYPE">path</tt></td>
          <td>16+16n bytes</td>
          <td>Closed path (similar to polygon)</td>
          <td>((x1,y1),...)</td>
        </tr>
        <tr>
          <td><tt class="TYPE">path</tt></td>
          <td>16+16n bytes</td>
          <td>Open path</td>
          <td>[(x1,y1),...]</td>
        </tr>
        <tr>
          <td><tt class="TYPE">polygon</tt></td>
          <td>40+16n bytes</td>
          <td>Polygon (similar to closed path)</td>
          <td>((x1,y1),...)</td>
        </tr>
        <tr>
          <td><tt class="TYPE">circle</tt></td>
          <td>24 bytes</td>
          <td>Circle</td>
          <td>&lt;(x,y),r&gt; (center point and radius)</td>
        </tr>
      </tbody>
    </table>

In [ ]:
%%sql
CREATE TABLE mygeom (
    p point,
    mbr box
);

In [ ]:
%sql INSERT INTO mygeom (p, mbr) VALUES ( point(1, 2), box(point(0, 4), point(0, 5)));

In [ ]:
%sql INSERT INTO mygeom (p, mbr) VALUES ( point '7, 9', box '((3, 5), (-20, -2))');

## Geometric functions

https://www.postgresql.org/docs/current/static/functions-geometry.html

<table class="table" summary="Geometric Operators" border="1">
<colgroup>
<col>
<col>
<col></colgroup>
<thead>
<tr>
<th>Operator</th>
<th>Description</th>
<th>Example</th>
</tr>
</thead>
<tbody>
<tr>
<td><code class="literal">+</code></td>
<td>Translation</td>
<td><code class="literal">box '((0,0),(1,1))' + point
'(2.0,0)'</code></td>
</tr>
<tr>
<td><code class="literal">-</code></td>
<td>Translation</td>
<td><code class="literal">box '((0,0),(1,1))' - point
'(2.0,0)'</code></td>
</tr>
<tr>
<td><code class="literal">\*</code></td>
<td>Scaling/rotation</td>
<td><code class="literal">box '((0,0),(1,1))' \* point
'(2.0,0)'</code></td>
</tr>
<tr>
<td><code class="literal">/</code></td>
<td>Scaling/rotation</td>
<td><code class="literal">box '((0,0),(2,2))' / point
'(2.0,0)'</code></td>
</tr>
<tr>
<td><code class="literal">#</code></td>
<td>Point or box of intersection</td>
<td><code class="literal">box '((1,-1),(-1,1))' # box
'((1,1),(-2,-2))'</code></td>
</tr>
<tr>
<td><code class="literal">#</code></td>
<td>Number of points in path or polygon</td>
<td><code class="literal"># path '((1,0),(0,1),(-1,0))'</code></td>
</tr>
<tr>
<td><code class="literal">@-@</code></td>
<td>Length or circumference</td>
<td><code class="literal">@-@ path '((0,0),(1,0))'</code></td>
</tr>
<tr>
<td><code class="literal">@@</code></td>
<td>Center</td>
<td><code class="literal">@@ circle '((0,0),10)'</code></td>
</tr>
<tr>
<td><code class="literal">##</code></td>
<td>Closest point to first operand on second operand</td>
<td><code class="literal">point '(0,0)' ## lseg
'((2,0),(0,2))'</code></td>
</tr>
<tr>
<td><code class="literal">&lt;-&gt;</code></td>
<td>Distance between</td>
<td><code class="literal">circle '((0,0),1)' &lt;-&gt; circle
'((5,0),1)'</code></td>
</tr>
<tr>
<td><code class="literal">&amp;&amp;</code></td>
<td>Overlaps? (One point in common makes this true.)</td>
<td><code class="literal">box '((0,0),(1,1))' &amp;&amp; box
'((0,0),(2,2))'</code></td>
</tr>
<tr>
<td><code class="literal">@&gt;</code></td>
<td>Contains?</td>
<td><code class="literal">circle '((0,0),2)' @&gt; point
'(1,1)'</code></td>
</tr>
<tr>

<td><code class="literal">&lt;@</code></td>
<td>Contained in or on?</td>
<td><code class="literal">point '(1,1)' &lt;@ circle
'((0,0),2)'</code></td>
</tr>
</tbody>
</table>

In [ ]:
%sql SELECT @@mbr AS center FROM mygeom;

In [ ]:
%sql SELECT p <-> mbr AS distance FROM mygeom;

## Exercise on geometric data

1. Translate the box (1, 1) (2, 2) two units to the left.
2. Find the intersection between the box (1, 0), (10, 10) and the box (1, 1), (2, 2)
3. Find the distance between the point (0, 2) and the box (1, 3), (-20, 0)
4. Is the point (7, 7) within the circle with the center (4, 2) and radius 5.83?

# Denormalisation

Fully normalised schemas have a high number of tables and references (foreign keys) between them

Can cause problems

  * Heavy join queries
  * Lots of locking constraints on foreign keys
  
This is typically not a problem in PostgreSQL

  * Support for row-level locking

## When and why to use denormalisation

* Query performance 
* Cached numbers
* Maintaining history




**Only** do this when you need it (and can prove that you need it)

## Method 1: Breaking functional dependencies

* When you denormalise you typically break functional dependencies
  * When done correctly, this is the same as implementing a cache
  * Can also be done with views!
 
* When done correctly, you have some kind of cache invalidation mechanism
  * Prevents inconsistencies

## Method 2: Denormalising with data types

* Arrays
  * ``int[]``
* Composite types
  * ``tuple``, ``point``, ``daterange``
* Enums
* XML
* JSON

## Method 3: Partitioning tables

* Tables can be split based on an attribute (date, id, etc.)

https://www.postgresql.org/docs/10/static/ddl-partitioning.html

1. Create table with partitioning

          CREATE TABLE measurement (
              logdate         date not null,
              ...
          ) PARTITION BY RANGE (logdate);
    
    
2. Create the partitions

        CREATE TABLE measurement_y2006m02 PARTITION OF measurement
            FOR VALUES FROM ('2006-02-01') TO ('2006-03-01');

3. Index (you want this)

        CREATE INDEX ON measurement_y2006m02 (logdate);

Befor partitioning    
    
    EXPLAIN SELECT count(*) FROM measurement WHERE logdate >= DATE '2008-01-01';
                                              QUERY PLAN
    -----------------------------------------------------------------------------------------------
     Aggregate  (cost=158.66..158.68 rows=1 width=0)
       ->  Append  (cost=0.00..151.88 rows=2715 width=0)
             ->  Seq Scan on measurement  (cost=0.00..30.38 rows=543 width=0)
                   Filter: (logdate >= '2008-01-01'::date)
             ->  Seq Scan on measurement_y2006m02 measurement  (cost=0.00..30.38 rows=543 width=0)
                   Filter: (logdate >= '2008-01-01'::date)
             ->  Seq Scan on measurement_y2006m03 measurement  (cost=0.00..30.38 rows=543 width=0)
                   Filter: (logdate >= '2008-01-01'::date)
    ...
             ->  Seq Scan on measurement_y2007m12 measurement  (cost=0.00..30.38 rows=543 width=0)
                   Filter: (logdate >= '2008-01-01'::date)
             ->  Seq Scan on measurement_y2008m01 measurement  (cost=0.00..30.38 rows=543 width=0)
                   Filter: (logdate >= '2008-01-01'::date)

After partitioning


    SET constraint_exclusion = on;
    EXPLAIN SELECT count(*) FROM measurement WHERE logdate >= DATE '2008-01-01';
                                              QUERY PLAN
    -----------------------------------------------------------------------------------------------
     Aggregate  (cost=63.47..63.48 rows=1 width=0)
       ->  Append  (cost=0.00..60.75 rows=1086 width=0)
             ->  Seq Scan on measurement  (cost=0.00..30.38 rows=543 width=0)
                   Filter: (logdate >= '2008-01-01'::date)
             ->  Seq Scan on measurement_y2008m01 measurement  (cost=0.00..30.38 rows=543 width=0)
                   Filter: (logdate >= '2008-01-01'::date)

## Other methods

* PostgreSQL has a number of extensions
  * hstore
  * ltree 
  * intarray
  * pg_trgm

## Denormalisation exercise


This is your shemas:

    Projects(ProjectId, ProjectName, CustomerId) 
     ProjectDetails(ProjectId, ItemId, ActivityId, EmployeeId, WorkDate, TimeSpent) 
     Customers(CustomerId, CustomerName) Activities(ActivityId, ActivityName) 
     Employees(EmployeeId, EmployeeName)
       
You have performance problems with a report that calculates total time spent on a project. How can you improve report performance? 

    Projects(ProjectId, ProjectName, CustomerId, StartDate, CustomerName, TotalTimeSpent)

More denormalisation exercises: https://msdn.microsoft.com/en-us/library/cc505841.aspx

## PL introduction

So far we have only talked about RDMSs as data storage. They can do so much more

* ``PL`` $=$ procedural language
* Comes in many flavours
  * PL/Python
  * PL/v8
  * PL/perl
  * PL/pgSQL
* PL/pgSQL is the shipped by default

In [186]:
%%sql
DO $$ 
BEGIN
    PERFORM '1';
END
$$ LANGUAGE plpgsql;

Done.


[]

## Triggers and functions in the bigger picture

* PL/pgSQL supports many more features
  * Caching
  * Channel/socket communication/notification
  * Event-based triggering
  
* Logic in the database versus logic in the application
    * Especially Oracle allows for program logic **within** the database

* With RDBMS, who should be responsible of updating the relations?

## Object-relational mapping

* ORM is a technique to create a direct mapping between the object in your OOP language and the object in your database
  * Maintains a 1:1 relation between the state of the objects in memory and in the database
  
* Popular frameworks that supports ORM
  * [JPA](https://en.wikipedia.org/wiki/Java_Persistence_API), Java
  * [Hibernate](http://hibernate.org/orm/what-is-an-orm/), Java
  * [LINQ](https://en.wikipedia.org/wiki/Language_Integrated_Query#LINQ_to_SQL), C#
  * [Zend](https://en.wikipedia.org/wiki/Zend_Framework), PHP
  * <a href="https://en.wikipedia.org/wiki/Django_(web_framework)">Django</a>, Python
  
* Problem to solve: RDBMS works with scalar values

## A word of warning

There’s  a  fundamental  difference  application state and database model

* The application implements workflows and interaction with the users (dynamic)
* The database model ensures a consistent view of world (static)






* The application model code should focus around user stories and interaction
* The database model should focus on keeping data consistent and safe

If you try to merge the two, you will feel pain.

## SQLAlchemy

In Python, SQL alchemy is a popular framework to achieve this. 

Models are defined in Python as classes.

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine
engine = create_engine('postgresql://appdev@data/appdev', echo=True)

### Creating a model in SQLAlchemy
    class Movie(Base):
        __tablename__ = 'movies'

        id = Column(Integer, primary_key=True)
        title = Column(String(255), nullable=False)
        year = Column(Integer)
        directed_by = Column(Integer, ForeignKey('directors.id'))

        director = relation("Director", backref='movies', lazy=False)

        def __init__(self, title=None, year=None):
            self.title = title
            self.year = year

        def __repr__(self):
            return "Movie(%r, %r, %r)" % (self.title, self.year, self.director)

## Storing data in SQLAlchemy

    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        session.add(m1)
        session.commit()
    except:
        session.rollback()

## Querying in SQLAlchemy

    alldata = session.query(Movie).all()
    for somedata in alldata:
        print somedata


Translates to:

    SELECT movies.id, movies.title, movies.year, movies.directed_by, directors.id, directors.name
    FROM movies LEFT OUTER JOIN directors ON directors.id = movies.directed_by

## Exercise on SQLAlchemy ORM

http://docs.sqlalchemy.org/en/latest/orm/tutorial.html

# Assignment 8: Spatial data and Magic: The gathering



## Introduction to JSON functions

https://www.postgresql.org/docs/current/static/functions-json.html

In [ ]:
%sql SELECT data FROM cards LIMIT 10;

<table><colgroup>
<col>
<col>
<col>
<col>
<col></colgroup>
<thead>
<tr>
<th>Operator</th>
<th>Right Operand Type</th>
<th>Description</th>
<th>Example</th>
<th>Example Result</th>
</tr>
</thead>
<tbody>
<tr>
<td><code class="literal">-&gt;</code></td>
<td><code class="type">int</code></td>
<td>Get JSON array element (indexed from zero, negative integers
count from the end)</td>
<td><code class="literal">'[{"a":"foo"},{"b":"bar"},{"c":"baz"}]'::json-&gt;2</code></td>
<td><code class="literal">{"c":"baz"}</code></td>
</tr>
<tr>
<td><code class="literal">-&gt;</code></td>
<td><code class="type">text</code></td>
<td>Get JSON object field by key</td>
<td><code class="literal">'{"a":
{"b":"foo"}}'::json-&gt;'a'</code></td>
<td><code class="literal">{"b":"foo"}</code></td>
</tr>
<tr>
<td><code class="literal">-&gt;&gt;</code></td>
<td><code class="type">int</code></td>
<td>Get JSON array element as <code class="type">text</code></td>
<td><code class="literal">'[1,2,3]'::json-&gt;&gt;2</code></td>
<td><code class="literal">3</code></td>
</tr>
<tr>
<td><code class="literal">-&gt;&gt;</code></td>
<td><code class="type">text</code></td>
<td>Get JSON object field as <code class="type">text</code></td>
<td><code class="literal">'{"a":1,"b":2}'::json-&gt;&gt;'b'</code></td>
<td><code class="literal">2</code></td>
</tr>
<tr>
<td><code class="literal">#&gt;</code></td>
<td><code class="type">text[]</code></td>
<td>Get JSON object at specified path</td>
<td><code class="literal">'{"a": {"b":{"c":
"foo"}}}'::json#&gt;'{a,b}'</code></td>
<td><code class="literal">{"c": "foo"}</code></td>
</tr>
<tr>
<td><code class="literal">#&gt;&gt;</code></td>
<td><code class="type">text[]</code></td>
<td>Get JSON object at specified path as <code class="type">text</code></td>
<td><code class="literal">'{"a":[1,2,3],"b":[4,5,6]}'::json#&gt;&gt;'{a,2}'</code></td>
<td><code class="literal">3</code></td>
</tr>
</tbody>
</table>

In [ ]:
%sql SELECT data -> 'type' FROM cards LIMIT 10;

<table>
<colgroup>
<col>
<col>
<col>
<col></colgroup>
<thead>
<tr>
<th>Operator</th>
<th>Right Operand Type</th>
<th>Description</th>
<th>Example</th>
</tr>
</thead>
<tbody>
<tr>
<td><code class="literal">@&gt;</code></td>
<td><code class="type">jsonb</code></td>
<td>Does the left JSON value contain the right JSON path/value
entries at the top level?</td>
<td><code class="literal">'{"a":1, "b":2}'::jsonb @&gt;
'{"b":2}'::jsonb</code></td>
</tr>
<tr>
<td><code class="literal">&lt;@</code></td>
<td><code class="type">jsonb</code></td>
<td>Are the left JSON path/value entries contained at the top level
within the right JSON value?</td>
<td><code class="literal">'{"b":2}'::jsonb &lt;@ '{"a":1,
"b":2}'::jsonb</code></td>
</tr>
<tr>
<td><code class="literal">?</code></td>
<td><code class="type">text</code></td>
<td>Does the <span class="emphasis"><em>string</em></span> exist as
a top-level key within the JSON value?</td>
<td><code class="literal">'{"a":1, "b":2}'::jsonb ? 'b'</code></td>
</tr>
<tr>
<td><code class="literal">?|</code></td>
<td><code class="type">text[]</code></td>
<td>Do any of these array <span class="emphasis"><em>strings</em></span> exist as top-level keys?</td>
<td><code class="literal">'{"a":1, "b":2, "c":3}'::jsonb ?|
array['b', 'c']</code></td>
</tr>
<tr>
<td><code class="literal">?&amp;</code></td>
<td><code class="type">text[]</code></td>
<td>Do all of these array <span class="emphasis"><em>strings</em></span> exist as top-level keys?</td>
<td><code class="literal">'["a", "b"]'::jsonb ?&amp; array['a',
'b']</code></td>
</tr>
</tbody>
</table>

In [ ]:
%sql SELECT data FROM cards WHERE data @> '{"type":"Enchantment"}' LIMIT 10;

# Assignment 8: Spatial data and Magic: The gathering

**Deadline**: 10th of April 12:00

**Review deadline**: 11th of April 23:59

This assignment is split into two parts: a spatial part and a JSON parsing part

1. In the first part you'll be processing the data in the ``geonames`` table.
  1. Find the tuple with the largest population
  2. Now calculate the distance from all locations to the tuple above using the ``<->`` function
  3. Now calculate the distance from all locations to the tuple above using the haversine distance
    1. The haversine distance has been implemented in PL/PGSQL here: https://gist.github.com/carlzulauf/1724506
  4. Plot the latitude and longitude of the largest 1000 populations on a 2-D scatter plot
    1. You can use [IPython-SQL](https://github.com/catherinedevlin/ipython-sql#graphing) or [Pandas](https://pandas.pydata.org/pandas-docs/stable/visualization.html) to create the plot
  
2. In the second part your job is to work with JSON data from the ``magic`` schema. The schema is based on the game Magic: The gathering
  1. Find all the cards that is made by Rob Alexander
  2. Find all the cards that have the subtype "Human"
  3. Group all the cards based on the colour. Aggregate them by counting the number of cards per colour.
  
Hand-in: a 2-d plot of the latitude and longitudes of the 1000 largest populations from the ``geonames`` table and a histogram that counts the number of cards for each colour